<a href="https://colab.research.google.com/github/claudiomiromarques/ConversorDeMoedasDesafioAluraOne/blob/main/Aline_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?'
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no Instagram, LinkedIn, Twitter e Facebook da Alura, pois eles costumam divulgar os eventos por lá.
*   **Assinar a newsletter da Alura:** Assim, você receberá informações sobre os próximos cursos e imersões diretamente no seu e-mail.
*   **Consultar a página de eventos da Alura:** Verifique regularmente a seção de eventos e imersões no site da Alura.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto
# Inserir a tool de busca do Google ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
      description="Agente que busca informações no Google.",
      tools=[google_search],
  )
  entrada_do_agente_buscador =  f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  lancamentos = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )
    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico.")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha! Vamos então criar o post sobre novidades em Agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos de agentes de IA mais relevantes e recentes, vou usar o Google Search para procurar notícias e artigos sobre o tema, focando nos lançamentos do último mês.
> 
> 
> Com base nas notícias e artigos de 2025, aqui estão alguns dos lançamentos e desenvolvimentos mais relevantes no campo dos agentes de IA:
> 
> 1.  **IBM watsonx Orchestrate:** A IBM anunciou atualizações durante a conferência IBM THINK em maio de 2025, oferecendo soluções para integrar, personalizar e implantar agentes. Inclui agentes pré-construídos para RH, vendas e compras, com integração a mais de 80 aplicações empresariais. [4] O Agent Catalog simplifica a implantação.
> 
> 2.  **Microsoft Security Copilot:** Em março de 2025, a Microsoft revelou novos agentes para o Security Copilot, projetados para responder a incidentes de segurança de forma mais rápida e precisa. [7] Esses agentes aprendem com o feedback das equipes e se adaptam aos fluxos de trabalho.
> 
> 3.  **Google Agent2Agent (A2A):** No Cloud Next 2025, o Google apresentou o protocolo A2A, que permite a colaboração entre agentes de IA de diferentes plataformas para automatizar fluxos empresariais. [9]
> 
> 4.  **OpenAI Guia para Construção de Agentes de IA:** A OpenAI publicou um guia prático sobre como construir agentes de IA autônomos, que utilizam modelos de linguagem para tomar decisões e executar tarefas, acessando ferramentas externas como navegadores ou APIs. [11]
> 
> 5.  **Llama 4 da Meta AI:** A Meta AI lançou a família de modelos Llama 4, incluindo o Llama 4 Scout, que possui uma janela de contexto de 10 milhões de tokens, permitindo analisar documentos longos. [12]
> 
> Outras tendências e desenvolvimentos notáveis:
> 
> *   **Crescimento do Mercado de Agentes de IA:** O mercado global de agentes de IA deverá atingir 52,6 bilhões de dólares até 2030, impulsionado pela necessidade de automação e decisões mais rápidas. [15]
> *   **Adoção Empresarial:** Estima-se que 25% das empresas adotarão agentes inteligentes em 2025, aumentando para 50% até 2027. [5]
> *   **Integração em Vários Setores:** Agentes de IA estão sendo incorporados em setores como e-commerce, atendimento ao cliente, marketing, finanças, bancos de investimento e saúde. [3, 15]
> *   **IA Generativa e Automação:** A IA generativa terá um impacto profundo no mercado de trabalho, com empresas investindo fortemente no aprimoramento desses sistemas para automação e análise de dados em tempo real. [3]
> 
> Esses lançamentos e tendências indicam um crescimento significativo no uso e desenvolvimento de agentes de IA, com um foco crescente em automação, integração em diversas áreas de negócios e avanços tecnológicos que permitem capacidades mais complexas e eficientes.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos e desenvolvimentos recentes no campo de Agentes de IA, vou criar um plano de conteúdo com foco nos seguintes tópicos, buscando informações adicionais para enriquecer o material:
> 
> 
> Com base nas minhas pesquisas, o tópico mais relevante e promissor para um post sobre Agentes de IA é o **Google Agent2Agent (A2A)**.
> 
> **Tema:** Google Agent2Agent (A2A): O Protocolo para Interoperabilidade de Agentes de IA
> 
> **Pontos Relevantes:**
> 
> *   **Interoperabilidade:** O A2A é um protocolo aberto que permite a comunicação e colaboração entre agentes de IA construídos em diferentes plataformas e por diferentes fornecedores.
> *   **Padrão Aberto:** O A2A é um padrão aberto, o que significa que qualquer pessoa pode usá-lo e contribuir para o seu desenvolvimento.
> *   **Casos de Uso:** O A2A pode ser usado para automatizar fluxos de trabalho complexos que exigem a colaboração entre vários agentes.
> *   **Segurança:** O A2A permite que os agentes interajam sem compartilhar informações confidenciais, como memória interna, ferramentas ou lógica proprietária.
> *   **Complementaridade com MCP:** O A2A complementa o Model Context Protocol (MCP), que permite que os agentes acessem ferramentas, APIs e recursos externos.
> *   **Suporte a Tarefas de Longa Duração:** O A2A foi projetado para ser flexível e oferecer suporte a cenários nos quais ele se destaca na execução de tarefas que vão desde ações rápidas até pesquisas aprofundadas que podem levar horas ou até mesmo dias quando há envolvimento humano. Ao longo desse processo, o A2A pode fornecer feedback em tempo real, notificações e atualizações de status aos usuários.
> *   **Princípios de Design:** O A2A foi projetado com base em cinco princípios fundamentais para realizar tarefas para usuários finais sem compartilhar memória, pensamentos ou ferramentas. Esses cinco princípios incluem: Adote os recursos agênticos: Os agentes colaboram sem compartilhar memória.
> 
> **Plano para o Post:**
> 
> 1.  **Introdução:**
>     *   Explicar o que são agentes de IA e por que eles são importantes.
>     *   Apresentar o problema da falta de interoperabilidade entre agentes de IA.
>     *   Introduzir o Google Agent2Agent (A2A) como uma solução para esse problema.
> 2.  **O que é o Google Agent2Agent (A2A)?**
>     *   Definir o A2A como um protocolo aberto para interoperabilidade de agentes de IA.
>     *   Explicar os objetivos do A2A:
>         *   Permitir que agentes de diferentes plataformas se comuniquem e colaborem.
>         *   Automatizar fluxos de trabalho complexos.
>         *   Garantir a segurança e a privacidade dos dados.
>     *   A2A foi projetado com base em cinco princípios fundamentais para realizar tarefas para usuários finais sem compartilhar memória, pensamentos ou ferramentas
> 3.  **Como o A2A Funciona?**
>     *   Descrever os principais componentes do A2A:
>         *   Protocolo de comunicação.
>         *   Formato de mensagem.
>         *   Serviço de descoberta de agentes.
>     *   Explicar como os agentes usam o A2A para se comunicar e colaborar.
>         *   Atores e Protocolos:
>         *   O protocolo A2A define três atores principais: Usuário: O usuário final que inicia uma tarefa. Agente cliente: O solicitante que formula e envia uma tarefa em nome do usuário. Agente remoto: O agente destinatário que executa a tarefa e envia os resultados.
>         *   Arquitetura e ciclo de vida da comunicação do agente.
> 4.  **Casos de Uso do A2A:**
>     *   Apresentar exemplos de como o A2A pode ser usado em diferentes setores:
>         *   E-commerce: Agentes de recomendação de produtos que colaboram para oferecer sugestões mais personalizadas.
>         *   Atendimento ao cliente: Agentes de chatbot que transferem conversas para agentes humanos quando necessário.
>         *   Segurança cibernética: Agentes de detecção de ameaças que compartilham informações sobre novos ataques.
> 5.  **Benefícios do A2A:**
>     *   Listar os benefícios do A2A para empresas e desenvolvedores:
>         *   Maior interoperabilidade entre agentes de IA.
>         *   Automação de fluxos de trabalho mais complexos.
>         *   Melhor segurança e privacidade dos dados.
>         *   Inovação acelerada no campo da IA.
>         *   Suporte a tarefas de longa duração, fornecendo feedback em tempo real, notificações e atualizações de status aos usuários.
> 6.  **Como Começar a Usar o A2A:**
>     *   Fornecer informações sobre como começar a usar o A2A:
>         *   Documentação do A2A.
>         *   Exemplos de código.
>         *   Comunidade A2A.
>     *   Incentivar os leitores a experimentar o A2A e contribuir para o seu desenvolvimento.
> 7.  **Conclusão:**
>     *   Reafirmar a importância da interoperabilidade de agentes de IA.
>     *   Destacar o potencial do Google Agent2Agent (A2A) para transformar o campo da IA.
>     *   Incentivar os leitores a explorar o A2A e a se juntar à comunidade.
> 
> Este plano fornecerá uma base sólida para um post informativo e envolvente sobre o Google Agent2Agent (A2A).
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🚀 Já ouviu falar em agentes de IA? Eles são o futuro da automação e da inteligência artificial! 🤖
> 
> Mas imagine um mundo onde cada agente fala uma língua diferente… 🗣️🤯 É aí que entra o **Google Agent2Agent (A2A)**, o protocolo que vai unificar essa galera!
> 
> O A2A é como um tradutor universal para agentes de IA, permitindo que eles se comuniquem e trabalhem juntos, não importa a plataforma. 🤝 Isso significa fluxos de trabalho mais inteligentes, seguros e eficientes.
> 
> **Como funciona?** 🤔
> O A2A estabelece um padrão aberto para que os agentes troquem informações sem revelar dados sensíveis. É como se eles pudessem colaborar sem mostrar a receita secreta! 🤫
> 
> **Onde podemos usar?** 📍
> E-commerce, atendimento ao cliente, segurança cibernética… As possibilidades são infinitas! 🚀
> 
> Quer saber mais sobre essa revolução? Deixe um comentário e vamos juntos desvendar o futuro da IA! 💬
> 
> #InteligenciaArtificial #AgentesDeIA #GoogleA2A #InovacaoAlura
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais atrativo e claro para o público do Instagram. Aqui estão algumas sugestões:
> 
> **Problemas e Sugestões:**
> 
> *   **Tom:** Embora o tom já seja adequado, podemos adicionar ainda mais entusiasmo e gírias leves para engajar o público jovem.
> *   **Chamada para Ação:** A chamada para ação ("Deixe um comentário") é boa, mas podemos personalizá-la mais.
> *   **Hashtags:** Podemos adicionar algumas hashtags mais específicas e populares.
> 
> **Rascunho Revisado:**
> 
> 🚀 MANO, JÁ ERA! Os agentes de IA chegaram pra dominar o mundo da automação! 🤖
> 
> Imagina só: cada um desses caras falando uma língua diferente... Que bad, né? 🗣️🤯 É aí que o **Google Agent2Agent (A2A)** entra pra salvar o dia, unindo essa galera toda!
> 
> O A2A é tipo um tradutor simultâneo de primeira, permitindo que os agentes de IA se entendam e trabalhem juntos, independente da plataforma. 🤝 Imagina os fluxos de trabalho super inteligentes, seguros e eficientes!
> 
> **Como que faz essa mágica?** 🤔
> O A2A cria um padrão aberto pra que os agentes troquem informações sem expor os podres. É tipo uma collab secreta, saca? 🤫
> 
> **Onde essa parada vai bombar?** 📍
> E-commerce, atendimento ao cliente, segurança cibernética… Se liga, o céu é o limite! 🚀
> 
> Curtiu a brisa e quer saber mais sobre essa revolução? Manda um direct e vamo decifrar juntos o futuro da IA! 💬
> 
> #InteligenciaArtificial #AgentesDeIA #GoogleA2A #Inovacao #Tech #FuturoDaTecnologia


--------------------------------------------------------------
